In [2]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
!pip install holidays

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import holidays
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.stats import  randint
from scipy.stats import uniform

ModuleNotFoundError: No module named 'holidays'

## Build X_train

In [39]:
X_train = pd.read_csv('fl_train')

In [40]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12741988 entries, 0 to 12741987
Data columns (total 21 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Unnamed: 0           int64  
 1   Unnamed: 0.1         int64  
 2   fl_date              object 
 3   op_unique_carrier    object 
 4   tail_num             object 
 5   op_carrier_fl_num    int64  
 6   origin_airport_id    int64  
 7   dest_airport_id      int64  
 8   crs_dep_time         int64  
 9   dep_delay            float64
 10  crs_arr_time         int64  
 11  arr_delay            float64
 12  cancelled            float64
 13  cancellation_code    object 
 14  crs_elapsed_time     float64
 15  distance             float64
 16  carrier_delay        float64
 17  weather_delay        float64
 18  nas_delay            float64
 19  security_delay       float64
 20  late_aircraft_delay  float64
dtypes: float64(10), int64(7), object(4)
memory usage: 2.0+ GB


In [43]:
X_train = X_train[['fl_date','op_unique_carrier','op_carrier_fl_num','origin_airport_id','crs_dep_time','dest_airport_id','crs_arr_time','distance','arr_delay']]

In [76]:
X_train.head()

,distance,uni_fl,arr_airport_hour,dep_airport_hour,is_holiday,month_01,month_02,month_03,month_04,month_05,...,month_10,month_11,month_12,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
0,594.0,11.047445,4.317572,12.011971,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1,644.0,18.049091,20.375983,7.351504,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,930.0,6.080851,-0.549418,8.711279,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
3,453.0,-2.385633,-2.763755,-1.809239,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,342.0,3.357506,9.322371,9.268407,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [49]:
# after test data, arr_delay should be 0. 
X_train['arr_delay'].fillna(0,inplace=True)

#remove outliner
X_train = X_train[(X_train['arr_delay'] > -80) & (X_train['arr_delay'] <= 480)] 

#combine features
def combine_str_num(x,y):
    return x + str(y)
X_train['uni_fl'] = np.vectorize(combine_str_num)(X_train['op_unique_carrier'],X_train['op_carrier_fl_num'])
X_train.drop(columns=['op_unique_carrier','op_carrier_fl_num'],inplace=True)

# feature coding
unifl_target_mean = X_train.groupby(['uni_fl'])['arr_delay'].mean()
X_train['uni_fl'] =X_train['uni_fl'].map(unifl_target_mean)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/pandas/core/frame.py:4908: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [54]:
## Combine the airportid and related hour
def airport_hour(id,t):
    '''
    combine airport and hour to a new field
    '''
    h = str(t)[:-2]
    if len(h) < 1:
        return str(id) + '24'
    elif len(h) < 2:
        return str(id) + '0' + h
    else:
        return str(id) + h
    
X_train['arr_airport_hour'] = np.vectorize(airport_hour)(X_train['dest_airport_id'],X_train['crs_arr_time'])
X_train['dep_airport_hour'] = np.vectorize(airport_hour)(X_train['origin_airport_id'],X_train['crs_dep_time'])

# feature coding
dep_hour_target_mean = X_train.groupby(['dep_airport_hour'])['arr_delay'].mean()
X_train['dep_airport_hour'] =X_train['dep_airport_hour'].map(dep_hour_target_mean)
arr_hour_target_mean = X_train.groupby(['arr_airport_hour'])['arr_delay'].mean()
X_train['arr_airport_hour'] =X_train['arr_airport_hour'].map(arr_hour_target_mean)

X_train.drop(columns=['dest_airport_id','crs_arr_time','origin_airport_id','crs_dep_time'],inplace=True)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [75]:
#transform fl_date
def getmonth(x):
    return x.split(sep='-')[1]
X_train['month'] = X_train['fl_date'].apply(getmonth)

def getdayofweek(x):
    year_s, mon_s, day_s = x.split('-')
    fl_d = datetime(int(year_s), int(mon_s), int(day_s))
    return fl_d.weekday()
X_train['day_of_week'] = X_train['fl_date'].apply(getdayofweek)

us_holidays = holidays.UnitedStates()
def isholiday(x):
    year_s, mon_s, day_s = x.split('-')
    if datetime(int(year_s), int(mon_s), int(day_s)) in us_holidays:
        return 1
    else:
        return 0
X_train['is_holiday'] = X_train['fl_date'].apply(isholiday)

y_train = X_train['arr_delay']

X_train = X_train.drop(columns=['fl_date','arr_delay'])

X_train = pd.get_dummies(X_train,columns=['month','day_of_week'])

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [18]:
#scaler = StandardScaler()
#X = scaler.fit_transform(X_train)

In [ ]:
#x_small = X[:100000]
#y_small = y_train[:100000]

### Build X_test

In [66]:
X_test = pd.read_csv('fl_test')

In [74]:
X_test.head()

,distance,uni_fl,arr_airport_hour,dep_airport_hour,is_holiday,month_01,month_02,month_03,month_04,month_05,...,month_10,month_11,month_12,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
0,55.0,2.786466,-1.767890,0.127554,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,430.0,5.369181,7.144641,11.648000,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,213.0,2.822811,3.456682,0.872361,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,342.0,0.000000,2.734760,4.133200,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,764.0,6.137931,19.311009,9.921022,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0


In [67]:
X_test = X_test[['fl_date','op_unique_carrier','op_carrier_fl_num','origin_airport_id','crs_dep_time','dest_airport_id','crs_arr_time','distance','arr_delay']]

In [69]:
# after test data, arr_delay should be 0. 
X_test['arr_delay'].fillna(0,inplace=True)

#remove outliner
#X_train = X_train[(X_train['arr_delay'] > -80) & (X_train['arr_delay'] < 180)] 

#combine features
def combine_str_num(x,y):
    return x + str(y)
X_test['uni_fl'] = np.vectorize(combine_str_num)(X_test['op_unique_carrier'],X_test['op_carrier_fl_num'])
X_test.drop(columns=['op_unique_carrier','op_carrier_fl_num'],inplace=True)

# feature coding
#unifl_target_mean = X_test.groupby(['uni_fl'])['arr_delay'].mean()
X_test['uni_fl'] =X_test['uni_fl'].map(unifl_target_mean)

In [ ]:
## Combine the airportid and related hour
def airport_hour(id,t):
    '''
    combine airport and hour to a new field
    '''
    h = str(t)[:-2]
    if len(h) < 1:
        return str(id) + '24'
    elif len(h) < 2:
        return str(id) + '0' + h
    else:
        return str(id) + h
    
X_test['arr_airport_hour'] = np.vectorize(airport_hour)(X_test['dest_airport_id'],X_test['crs_arr_time'])
X_test['dep_airport_hour'] = np.vectorize(airport_hour)(X_test['origin_airport_id'],X_test['crs_dep_time'])

# feature coding
#dep_hour_target_mean = X_train.groupby(['dep_airport_hour'])['arr_delay'].mean()
X_test['dep_airport_hour'] =X_test['dep_airport_hour'].map(dep_hour_target_mean)
#arr_hour_target_mean = X_train.groupby(['arr_airport_hour'])['arr_delay'].mean()
X_test['arr_airport_hour'] =X_test['arr_airport_hour'].map(arr_hour_target_mean)

X_test.drop(columns=['dest_airport_id','crs_arr_time','origin_airport_id','crs_dep_time'],inplace=True)

In [72]:
#transform fl_date
def getmonth(x):
    return x.split(sep='-')[1]
X_test['month'] = X_test['fl_date'].apply(getmonth)

def getdayofweek(x):
    year_s, mon_s, day_s = x.split('-')
    fl_d = datetime(int(year_s), int(mon_s), int(day_s))
    return fl_d.weekday()
X_test['day_of_week'] = X_test['fl_date'].apply(getdayofweek)

us_holidays = holidays.UnitedStates()
def isholiday(x):
    year_s, mon_s, day_s = x.split('-')
    if datetime(int(year_s), int(mon_s), int(day_s)) in us_holidays:
        return 1
    else:
        return 0
X_test['is_holiday'] = X_test['fl_date'].apply(isholiday)

y_test = X_test['arr_delay']



In [73]:
X_test = X_test.drop(columns=['fl_date','arr_delay'])

X_test = pd.get_dummies(X_test,columns=['month','day_of_week'])

#X_test = scaler.transform(X_test)

In [14]:
###  XGBoost Modeling

In [77]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [90]:
params = {
    # Parameters that we are going to tune.
    'max_depth':11,
    'min_child_weight': 5,
    'eta':.3,
    'subsample': 0.7,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
}

In [95]:
num_boost_round = 30

In [92]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=2
)

[0]	Test-rmse:49.19681
[1]	Test-rmse:48.94568
[2]	Test-rmse:48.81043
[3]	Test-rmse:48.73124
[4]	Test-rmse:48.68348
[5]	Test-rmse:48.65303
[6]	Test-rmse:48.63278
[7]	Test-rmse:48.61763
[8]	Test-rmse:48.60550
[9]	Test-rmse:48.59861
[10]	Test-rmse:48.59085
[11]	Test-rmse:48.58531
[12]	Test-rmse:48.58006
[13]	Test-rmse:48.57711
[14]	Test-rmse:48.57379
[15]	Test-rmse:48.57136
[16]	Test-rmse:48.56950
[17]	Test-rmse:48.56732
[18]	Test-rmse:48.56549
[19]	Test-rmse:48.56374


In [93]:
y_pred = model.predict(dtest)

In [94]:
## third round  48.67
r2_score(y_test,y_pred)

0.03500066824855497

In [21]:
## second round with arr_delay <480
#r2_score(y_test, y_pred)

0.02144653066658464

In [27]:
## first round with arr_delay < 180
#

0.013895135285908466

In [96]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(6,11)]
    for colsample in [i/10. for i in range(8,11)]
]

In [ ]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=3
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
	MAE 21.3599238 for 0 rounds
CV with subsample=1.0, colsample=0.9
	MAE 21.4174592 for 0 rounds
CV with subsample=1.0, colsample=0.8


In [ ]:
## first try on XGBoost 
#r2_score(y_test,y_pred)

-0.0005833909885790689

In [ ]:
randm_src = RandomizedSearchCV(estimator=model, param_distributions = parameters,
                               cv = 2, n_iter = 10, n_jobs=-1)